#### Low Level Design

##### Implement a stack with max API. [EPI: 8.01]

In [ ]:
class Stack:

    ElementWithCachedMax = namedtuple('ElementWithCachedMax', ('element', 'max'))

    def __init__(self):
        self._element_with_cached_max = []

    def empty(self):

        return len(self._element_with_cached_max) == 0

    def max(self):

        if self.empty():
            raise IndexError('max(): empty stack')
        return self._element_with_cached_max[-1].max

    def pop(self):

        if self.empty():
            raise IndexError('pop(): empty stack')
        return self._element_with_cached_max.pop().element

    def push(self, x):

        self._element_with_cached_max.append(
            self.ElementWithCachedMax(x, x if self.empty() else max(x, self.max()))
        )

In [ ]:
def _test_stack_with_max(ops):
    try:
        s = Stack()

        for (op, arg) in ops:
            if op == 'Stack':
                s = Stack()
            elif op == 'push':
                s.push(arg)
            elif op == 'pop':
                result = s.pop()
                if result != arg:
                    raise TestFailure("Pop: expected " + str(arg) + ", got " +
                                      str(result))
            elif op == 'max':
                result = s.max()
                if result != arg:
                    raise TestFailure("Max: expected " + str(arg) + ", got " +
                                      str(result))
            elif op == 'empty':
                result = int(s.empty())
                if result != arg:
                    raise TestFailure("Empty: expected " + str(arg) +
                                      ", got " + str(result))
            else:
                raise RuntimeError("Unsupported stack operation: " + op)
    except IndexError:
        raise TestFailure('Unexpected IndexError exception')

##### Implement a circular queue. [EPI: 8.07]

In [ ]:
class Queue:

    SCALE_FACTOR = 2

    def __init__(self, capacity):

        self._entries = [None] * capacity
        self._head = self._tail = self._num_queue_elements = 0

    def enqueue(self, x):

        if self._num_queue_elements == len(self._entries):  # Needs to resize.
            # Makes the queue elements appear consecutively.
            self._entries = (
                self._entries[self._head:] + self._entries[:self._head])
            # Resets head and tail.
            self._head, self._tail = 0, self._num_queue_elements
            self._entries += [None] * (
                len(self._entries) * Queue.SCALE_FACTOR - len(self._entries))

        self._entries[self._tail] = x
        self._tail = (self._tail + 1) % len(self._entries)
        self._num_queue_elements += 1

    def dequeue(self):

        if not self._num_queue_elements:
            raise IndexError('empty queue')
        self._num_queue_elements -= 1
        result = self._entries[self._head]
        self._head = (self._head + 1) % len(self._entries)
        return result

    def size(self):

        return self._num_queue_elements


def queue_tester(ops):
    q = Queue(1)

    for (op, arg) in ops:
        if op == 'Queue':
            q = Queue(arg)
        elif op == 'enqueue':
            q.enqueue(arg)
        elif op == 'dequeue':
            result = q.dequeue()
            if result != arg:
                raise TestFailure(
                    "Dequeue: expected " + str(arg) + ", got " + str(result))
        elif op == 'size':
            result = q.size()
            if result != arg:
                raise TestFailure(
                    "Size: expected " + str(arg) + ", got " + str(result))
        else:
            raise RuntimeError("Unsupported queue operation: " + op)


##### Implement a queue using stacks. [EPI: 8.08] | [neetcode](https://www.youtube.com/watch?v=eanwa3ht3YQ)

In [ ]:

class Queue:
    def __init__(self):
        self._enq, self._deq = [], []

    def enqueue(self, x):

        self._enq.append(x)

    def dequeue(self):

        if not self._deq:
            # Transfers the elements in _enq to _deq.
            while self._enq:
                self._deq.append(self._enq.pop())

        if not self._deq:  # _deq is still empty!
            raise IndexError('empty queue')
        return self._deq.pop()


def queue_tester(ops):
    from test_framework.test_failure import TestFailure

    try:
        q = Queue()

        for (op, arg) in ops:
            if op == 'Queue':
                q = Queue()
            elif op == 'enqueue':
                q.enqueue(arg)
            elif op == 'dequeue':
                result = q.dequeue()
                if result != arg:
                    raise TestFailure("Dequeue: expected " + str(arg) +
                                      ", got " + str(result))
            else:
                raise RuntimeError("Unsupported queue operation: " + op)
    except IndexError:
        raise TestFailure('Unexpected IndexError exception')


##### Implement a queue with max API. [EPI: 8.09]

In [ ]:
class QueueWithMax:
    def __init__(self):
        self._enqueue = Stack()
        self._dequeue = Stack()

    def enqueue(self, x):

        self._enqueue.push(x)

    def dequeue(self):

        if self._dequeue.empty():
            while not self._enqueue.empty():
                self._dequeue.push(self._enqueue.pop())
        if not self._dequeue.empty():
            return self._dequeue.pop()
        raise IndexError('Cannot get dequeue() on empty queue.')

    def max(self):

        if not self._enqueue.empty():
            return self._enqueue.max() if self._dequeue.empty() else max(
                self._enqueue.max(), self._dequeue.max())
        elif not self._dequeue.empty():
            return self._dequeue.max()
        raise IndexError('Cannot get max() on empty queue.')

In [ ]:
def queue_tester(ops):

    try:
        q = QueueWithMax()

        for (op, arg) in ops:
            if op == 'QueueWithMax':
                q = QueueWithMax()
            elif op == 'enqueue':
                q.enqueue(arg)
            elif op == 'dequeue':
                result = q.dequeue()
                if result != arg:
                    raise TestFailure("Dequeue: expected " + str(arg) +
                                      ", got " + str(result))
            elif op == 'max':
                result = q.max()
                if result != arg:
                    raise TestFailure(
                        "Max: expected " + str(arg) + ", got " + str(result))
            else:
                raise RuntimeError("Unsupported queue operation: " + op)
    except IndexError:
        raise TestFailure('Unexpected IndexError exception')

##### Implement an ISBN cache - [EPI:12.3]

The International Standard Book Number (ISBN) is a unique commercial book identifier. It is a string of length 10. The first 9 characters are digits; the last character is a check character. The check character is the sum of the first 9 digits, mod 11, with 10 represented by 'X'.(Modem ISBNs use 13 digits, and the check digit is taken mod 10; this problem is concerned
with 10-digit ISBNs.)

- Create a cache for looking up prices of books identified by their ISBN. You implement `lookup()`, `insert()`, and `remove()` methods.

- Use the Least Recently Used (LRU) policy for cache eviction. 
- If an ISBN is already present, insert should not change the price, but it should update
that entry to be the most recently used entry. 
- Lookup should also update that entry to be the most recently used entry.

Hint: Amortilze the cost of deletion. Altematively, use an auxiliary data structure.

In [ ]:
class LruCache:
    def __init__(self, capacity):
        self._isbn_price_table = OrderedDict()
        self._capacity = capacity

    def lookup(self, isbn):
        if isbn not in self._isbn_price_table: return -1
        price = self._isbn_price_table.pop(isbn)
        self._isbn_price_table[isbn] = price
        return price

    def insert(self, isbn, price):
        # We add the value for key only if key is not present - we don't update existing values.
        if isbn in self._isbn_price_table:
            price = self._isbn_price_table.pop(isbn)
        elif len(self._isbn_price_table) == self._capacity:
            self._isbn_price_table.popitem(last=False)
        
        self._isbn_price_table[isbn] = price

    def erase(self, isbn):
        return self._isbn_price_table.pop(isbn, None) is not None

##### [146. LRU Cache](https://leetcode.com/problems/lru-cache/description/) | [neetcode](https://www.youtube.com/watch?v=7ABFKPK2hD4&list=PLot-Xpze53lfOdF3KwpMSFEyfE77zIwiP&index=27)

Design a data structure that follows the constraints of a Least Recently Used (LRU) cache.

-   **Implement the LRUCache class**:

    -   `LRUCache(int capacity)` Initialize the LRU cache with positive size capacity.
    -   `get(int key) -> int` Return the value of the key if the key exists, otherwise return -1.
    -   `put(int key, int value) -> None` Update the value of the key if the key exists. Otherwise, add the key-value pair to the cache. If the number of keys exceeds the capacity from this operation, evict the least recently used key.
    -   The functions `get(..)` and `put(..)` must each run in `O(1)` average time complexity.

 

-   **Example 1**:

    -   `Input`: 
        -   ["LRUCache", "put", "put", "get", "put", "get", "put", "get", "get", "get"]
        -   [[2], [1, 1], [2, 2], [1], [3, 3], [2], [4, 4], [1], [3], [4]]
    `Output`: [null, null, null, 1, null, -1, null, -1, 3, 4]

    -   `Explanation:`
        ```python
        LRUCache lRUCache = new LRUCache(2);
        lRUCache.put(1, 1); # cache is {1=1}
        lRUCache.put(2, 2); # cache is {1=1, 2=2}
        lRUCache.get(1);    # return 1
        lRUCache.put(3, 3); # LRU key was 2, evicts key 2, cache is {1=1, 3=3}
        lRUCache.get(2);    # returns -1 (not found)
        lRUCache.put(4, 4); # LRU key was 1, evicts key 1, cache is {4=4, 3=3}
        lRUCache.get(1);    # return -1 (not found)
        lRUCache.get(3);    # return 3
        lRUCache.get(4);    # return 4
        ```

In [ ]:
class Node:
    def __init__(self, key, val):
        self.key, self.val = key, val
        self.prev = self.next = None


class LRUCache:
    def __init__(self, capacity: int):
        self.cap = capacity
        self.cache = {}  # map key to node

        self.left, self.right = Node(0, 0), Node(0, 0)
        self.left.next, self.right.prev = self.right, self.left

    # remove node from list
    def remove(self, node):
        prev, nxt = node.prev, node.next
        prev.next, nxt.prev = nxt, prev

    # insert node at right
    def insert(self, node):
        prev, nxt = self.right.prev, self.right
        prev.next = nxt.prev = node
        node.next, node.prev = nxt, prev

    def get(self, key: int) -> int:
        if key in self.cache:
            self.remove(self.cache[key])
            self.insert(self.cache[key])
            return self.cache[key].val
        return -1

    def put(self, key: int, value: int) -> None:
        if key in self.cache:
            self.remove(self.cache[key])
        self.cache[key] = Node(key, value)
        self.insert(self.cache[key])

        if len(self.cache) > self.cap:
            # remove from the list and delete the LRU from hashmap
            lru = self.left.next
            self.remove(lru)
            del self.cache[lru.key]

##### [380. Insert Delete GetRandom O(1)](https://leetcode.com/problems/insert-delete-getrandom-o1/description/) | [leetcode](https://www.youtube.com/watch?v=46dZH7LDbf8&t=155s)

Implement the RandomizedSet class:

-   `RandomizedSet()` -> Initializes the RandomizedSet object.
-   `bool insert(int val)` -> Inserts an item val into the set if not present. Returns true if the item was not present, false otherwise.
-   `bool remove(int val)` -> Removes an item val from the set if present. Returns true if the item was present, false otherwise.
-   `int getRandom()` -> Returns a random element from the current set of elements (it's guaranteed that at least one element exists when this method is called). Each element must have the same probability of being returned.

You must implement the functions of the class such that each function works in average O(1) time complexity.

 

-   `Example 1`:

-   `Input`:
    -   ["RandomizedSet", "insert", "remove", "insert", "getRandom", "remove", "insert", "getRandom"]
    -   [[], [1], [2], [2], [], [1], [2], []]
-   `Output`:
    -   [null, true, false, true, 2, true, false, 2]

-   `Explanation`:
    -   `RandomizedSet randomizedSet = new RandomizedSet();`
    -   `randomizedSet.insert(1);`    // Inserts 1 to the set. Returns true as 1 was inserted successfully.
    -   `randomizedSet.remove(2);`    // Returns false as 2 does not exist in the set.
    -   `randomizedSet.insert(2);`    // Inserts 2 to the set, returns true. Set now contains [1,2].
    -   `randomizedSet.getRandom();`  // getRandom() should return either 1 or 2 randomly.
    -   `randomizedSet.remove(1);`    // Removes 1 from the set, returns true. Set now contains [2].
    -   `randomizedSet.insert(2);`    // 2 was already in the set, so return false.
    -   `randomizedSet.getRandom();`  // Since 2 is the only number in the set, getRandom() will always return 2.
 

-   `Constraints`:

    -   $-2^{31} <= val <= 2^{31} - 1$
    -   At most 2 * 105 calls will be made to `insert`, `remove`, and `getRandom`.
    -   There will be at least one element in the data structure when `getRandom` is called.